In [2]:
# Import Librarys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Import datasets Titles movies

df_amazon = pd.read_csv("./platform_api/datasets/amazon_prime_titles.csv")
df_disney = pd.read_csv("./platform_api/datasets/disney_plus_titles.csv")
df_hulu = pd.read_csv("./platform_api/datasets/hulu_titles.csv")
df_netflix = pd.read_csv("./platform_api/datasets/netflix_titles.csv")


In [4]:
# Join Datasets titles movies

titles_movies = pd.concat([df_amazon, df_disney, df_hulu, df_netflix], ignore_index=True)
titles_movies.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...


In [5]:
titles_movies.shape

(22998, 12)

#### Transform Data

- Generar campo id: Cada id se compondrá de la primera letra del nombre de la plataforma, seguido del show_id ya presente en los datasets (ejemplo para títulos de Amazon = as123)

- Los valores nulos del campo rating deberán reemplazarse por el string “G” (corresponde al maturity rating: “general for all audiences”

- De haber fechas, deberán tener el formato AAAA-mm-dd

- Los campos de texto deberán estar en minúsculas, sin excepciones

- El campo duration debe convertirse en dos campos: duration_int y duration_type. El primero será un integer y el segundo un string indicando la unidad de medición de duración: min (minutos) o season (temporadas)

In [6]:
# transform rating null to G
titles_movies["rating"].fillna("G", inplace = True)

# verificate null 
titles_movies['rating'].isnull().sum()

# Pass column to datetime
titles_movies['date_added'] = pd.to_datetime(titles_movies['date_added'])

In [ ]:
# Duration to duration_int and duration_type
duration_new = titles_movies["duration"].str.split(" ", n = 1, expand = True)

titles_movies['duration_int'] = duration_new[0]

titles_movies['duration_type'] = duration_new[1]

titles_movies.drop(columns =["duration"], inplace = True)

In [13]:
# duration_type replace Season to season
titles_movies['duration_type'] = titles_movies['duration_type'].replace('Seasons', 'season')

In [14]:
#duration_int convert to Int
titles_movies['duration_int'] = titles_movies['duration_int'].fillna(0)
titles_movies['duration_int'] = titles_movies['duration_int'].astype('int')

In [15]:
#Convert Lowercase all Columns
titles_movies['title'] = titles_movies['title'].str.lower()
titles_movies['type'] = titles_movies['type'].str.lower()
titles_movies['director'] = titles_movies['director'].str.lower()
titles_movies['cast'] = titles_movies['cast'].str.lower()
titles_movies['country'] = titles_movies['country'].str.lower()
titles_movies['listed_in'] = titles_movies['listed_in'].str.lower()
titles_movies['description'] = titles_movies['description'].str.lower()
titles_movies['duration_type'] = titles_movies['duration_type'].str.lower()

In [16]:
# Import datasets

df1 = pd.read_csv('../ratings_datasets/1.csv')
df2 = pd.read_csv('../ratings_datasets/2.csv')
df3 = pd.read_csv('../ratings_datasets/3.csv')
df4 = pd.read_csv('../ratings_datasets/4.csv')
df5 = pd.read_csv('../ratings_datasets/5.csv')
df6 = pd.read_csv('../ratings_datasets/6.csv')
df7 = pd.read_csv('../ratings_datasets/7.csv')
df8 = pd.read_csv('../ratings_datasets/8.csv')

In [17]:
# Join Datasets users

users_id = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)
users_id.head(2)


,userId,rating,timestamp,movieId
0,1,1.0,1425941529,as680
1,1,4.5,1425942435,ns2186


In [12]:
users_id.shape

(11024289, 4)

In [13]:
# Rename columns rating to score
users_id = users_id.rename(columns={"rating": "score"})

In [14]:
# Rename columns movieId to Id
users_id = users_id.rename(columns={"movieId": 'id'})

In [15]:
# Sacamos el promedio por cada Id
promedios = users_id.groupby("id")["score"].mean()
promedios_redondeados = promedios.round(2)
score_medio = pd.DataFrame({'id': promedios_redondeados.index, 'score_medio': promedios_redondeados.values})

In [16]:
score_medio.shape

(22998, 2)

In [17]:
# getting the first letter

titles_movies['id'] = users_id['id'].str[0] + titles_movies['show_id']

#users_id['Platform'] = users_id['movieId'].str[0]

In [18]:
titles_movies.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,id
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,G,"comedy, drama",a small fishing village must procure a local d...,113,min,as1
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min,ns2


In [41]:
# Merge 
movies = pd.merge(titles_movies, score_medio, on='id', how='left')

In [42]:
movies.shape

(22998, 15)

In [ ]:
movies['platform'] = movies['id'].str[0]


In [25]:
movies['platform'] = movies['platform'].replace(['a'], 'amazon')


In [26]:
movies['platform'] = movies['platform'].replace(['d'], 'disney')

In [27]:
movies['platform'] = movies['platform'].replace(['n'], 'netflix')

In [28]:
movies['platform'] = movies['platform'].replace(['h'], 'hulu')

In [43]:
movies.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,duration_int,duration_type,id,score_medio
0,s1,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,G,"comedy, drama",a small fishing village must procure a local d...,113,min,as1,3.47
1,s2,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,110,min,ns2,3.55


In [24]:
movies.to_csv('movies_completo_api.csv')